<a href="https://colab.research.google.com/github/courtneyjhodgson/school/blob/main/Courtney_QDK_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# TODO: Put your names here.
# Name: Courtney Hodgson
# Name: Reese Feldmeier

In [ ]:
#Execute This Cell First! #This cell's contents should match those of photon.py

import random
import numpy as np

class InputError(Exception):
    def __int__(self, expression, message):
        self.expression = expression
        self.message = message

class Photon:

    def __init__(self, Hcomp=0, Vcomp=0):
        self.alpha = Hcomp
        self.beta  = Vcomp

    # This is for debugging purposes only!
    def toString(self):
        if np.isreal(self.alpha):
            string = str(self.alpha) + "|H> "
        else:
            string = str(self.alpha) + "|H> "
        if np.isreal(self.beta):
            if self.beta >= 0:
                string += "+ " + str(self.beta) + "|V>"
            else:
                string += "- " + str(-self.beta) + "|V>"
        else:
            string += "+ " + str(self.beta) + "|V>"
        return string

    def prepareVacuum(self):
        energyPerMode = 0.5; # in units of hbar*omega
        x0 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        y0 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        x1 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        y1 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        self.alpha = complex(x0, y0)
        self.beta  = complex(x1, y1)

    def prepare(self, alpha, beta, avgPhotonNumber):
        if avgPhotonNumber < 0:
            raise InputError()
        vac = Photon()
        vac.prepareVacuum()
        self.alpha = alpha * np.sqrt(avgPhotonNumber) + vac.alpha
        self.beta  = beta  * np.sqrt(avgPhotonNumber) + vac.beta

    def prepareH(self, avgPhotonNumber):
        self.prepare(1, 0, avgPhotonNumber)

    def prepareV(self, avgPhotonNumber):
        self.prepare(0,1, avgPhotonNumber)

    def prepareD(self, avgPhotonNumber):
        self.prepare(1/np.sqrt(2),  1/np.sqrt(2), avgPhotonNumber)

    def prepareA(self, avgPhotonNumber):
        self.prepare(1/np.sqrt(2), -1/np.sqrt(2), avgPhotonNumber)

    def prepareR(self, avgPhotonNumber):
        self.prepare(1/np.sqrt(2),  1j/np.sqrt(2), avgPhotonNumber)

    def prepareL(self, avgPhotonNumber):
        self.prepare(1/np.sqrt(2), -1j/np.sqrt(2), avgPhotonNumber)

    def measureHV(self, probDarkCount):
        if probDarkCount < 0 or probDarkCount > 1:
            raise InputError
        threshold  = -0.5*np.log(1 - np.sqrt(1-probDarkCount))
        intensityH = abs(self.alpha)**2
        intensityV = abs(self.beta)**2
        # The photon is absorbed by the detector:
        self.prepareVacuum()
        # The outcome is determined by threshold exceedances:
        if intensityH <= threshold and intensityV <= threshold:
            return "N" # no detection (invalid measurement)
        elif intensityH >  threshold and intensityV <= threshold:
            return "H" # single H photon detected
        elif intensityH <=  threshold and intensityV > threshold:
            return "V" # single V photon detected
        else:
            return "M" # multiple detections (invalid measurement)

    def measureDA(self, probDarkCount):
        a = self.alpha
        b = self.beta
        self.alpha = (a+b)/np.sqrt(2)
        self.beta  = (a-b)/np.sqrt(2)
        outcome = self.measureHV(probDarkCount)
        if outcome == "H": return "D"
        if outcome == "V": return "A"
        else: return outcome

    def measureRL(self, probDarkCount):
        a = self.alpha
        b = self.beta
        self.alpha = (a-b*1j)/np.sqrt(2)
        self.beta  = (a+b*1j)/np.sqrt(2)
        outcome = self.measureHV(probDarkCount)
        if outcome == "H": return "R"
        if outcome == "V": return "L"
        else: return outcome

    def applyPolarizer(self, theta, phi):
	# Apply a polarizing filter according to the input parameters.
	# theta=0,    phi=0:     H polarizer
	# theta=pi/2, phi=0:     V polarizer
	# theta=pi/4, phi=0:     D polarizer
	# theta=pi/4, phi=pi:    A polarizer
	# theta=pi/4, phi=+pi/2: R polarizer
	# theta=pi/4, phi=-pi/2: L polarizer
        z = np.exp(1j*phi)
        a = self.alpha
        b = self.beta
        self.alpha = a*(1+np.cos(2*theta))/2 + b*np.sin(2*theta)/2*np.conj(z)
        self.beta  = a*np.sin(2*theta)/2*z + b*(1-np.cos(2*theta))/2
        # Now add an extra vacuum component.
        vac = Photon()
        vac.prepareVacuum()
        a = vac.alpha
        b = vac.beta
        self.alpha = self.alpha + a*np.sin(theta)**2 + b*(-np.sin(2*theta)/2)*np.conj(z)
        self.beta  = self.beta  + a*(-np.sin(2*theta)/2)*z + b*np.cos(theta)**2

    def applyUnitaryGate(self, theta, phi, lamb):
        U = [[0,0],[0,0]]
        z1 = np.exp(1j*phi)
        z2 = -np.exp(1j*lamb)
        z3 = np.exp(1j*(lamb+phi))
        U[0][0] = np.cos(theta/2)
        U[1][0] = np.sin(theta/2)*z1
        U[0][1] = np.sin(theta/2)*z2
        U[1][1] = np.cos(theta/2)*z3
        a = self.alpha
        b = self.beta
        self.alpha = U[0][0]*a + U[0][1]*b
        self.beta  = U[1][0]*a + U[1][1]*b

    def applyXGate(self):
        # Applies the Pauli X gate
        self.applyUnitaryGate(np.pi, 0, np.pi)

    def applyYGate(self):
        # Applies the Pauli Y gate
        self.applyUnitaryGate(np.pi, np.pi/2, np.pi/2)

    def applyZGate(self):
        # Applies the Pauli X gate
        self.applyUnitaryGate(0, np.pi, 0)

    def applyHGate(self):
        # Applied the Hadamard (half-wavelength) gate
        self.applyUnitaryGate(np.pi/2, 0, np.pi)

    def applyQGate(self):
        # Applies the SH (quarter-wavelength) gate
        self.applyUnitaryGate(np.pi/2, np.pi/2, np.pi)

    def applyNoisyGate(self, p):
        # This operation acts as a depolarizing channel.
	# p = 0 leaves the photon unchanged.
	# p = 1 yields a completely random photon.
	# 0 < p < 1 yields a partially random photon.
        if p < 0 or p > 1:
            raise InputError
        theta = np.arccos(1 - 2*random.uniform(0,1)*p)
        phi   = p*(2*random.uniform(0,1) - 1)*np.pi
        lamb  = p*(2*random.uniform(0,1) - 1)*np.pi
        self.applyUnitaryGate(theta, phi, lamb)

    def applyAttenuation(self, r):
	# This operation acts as a partially reflecting beam splitter.
	# r = 0 leaves the photon unchanged.
	# r = 1 completely absorbs the photon, leaving a vacuum state.
	# 0 < r < 1 partially attenuates the photon and adds some vacuum.
	# r is the reflectivity.
        if r < 0 or r > 1:
            raise InputError
        t = np.sqrt(1-r*r) # t is the transmissivity.
        vac = Photon()
        vac.prepareVacuum()
        self.alpha = (self.alpha)*t + (vac.alpha)*r
        self.beta  = (self.beta )*t + (vac.beta)*r

In [ ]:
n = 100 # number of photons
p = 0.5 #average photon number
d = 0.5 #dark count probability

In [ ]:
# Alice --------------------------------------------

# Alice generates the raw key.
keyAlice = ""
for i in range(n): # Iterate over the number of photons.
    # Append a random character ('0' or '1') to the end.
    if random.randint(0,1)==0: # Flip a coin (0 or 1).
        keyAlice += '0'
    else:
        keyAlice += '1'
print("keyAlice    = " + keyAlice)

keyAlice    = 1000010011100100110110010100101010011111011011101100111011110000111100010001000100110101100101001101


In [ ]:
# Alice chooses the encoding basis for each key bit.
# This should be a string of '+'s and 'x's with '+'=H/V, 'x'=D/A.
basisAlice = ""
# TODO: Put your code here.
for i in range(n):
  if random.randint(0,1) == 0:
    basisAlice += '+'
  else:
    basisAlice += 'x'
  
print("basisAlice  = " + basisAlice)

basisAlice  = x++++++x+++xxxxxx++x+++x+x+x++x+xx+x++x++++++xxx++xxx+x++++x++x+x+xxxx+x+x+x+++xx++xx+++xxxx+x++xxxx


In [ ]:
# Alice selects a photon state according to the key and basis.
# This should be a string of the characters 'H', 'V', 'D', 'A'.
photonAlice = ""
# TODO: Put your code here.
for i in range(n):
  if keyAlice[i] == '0' and basisAlice[i] == '+':
    photonAlice += 'H'
  elif keyAlice[i] == '1' and basisAlice[i] == '+':
    photonAlice += 'V'
  elif keyAlice[i] == '0' and basisAlice[i] == 'x':
    photonAlice += 'D'
  else:
    photonAlice += 'A'

print("photonAlice  = " + photonAlice)

photonAlice  = AHHHHVHDVVVDDADDAVHAVHHAHAHDVHAHADHAVVAVHVVHVAADVVDDAVAHVVVAHHDHAVAADDHAHDHAHHHADHVADVHVADDAHAHHAADA


In [ ]:
# Alice prepares and sends each photon.
# Use the methods of the Photon class to prepare each photon.
photonArray = [Photon() for i in range(n)]

# TODO: Put your code here.
for i in range(n):
  if photonAlice[i] == 'H':
    photonArray[i].prepareH(p)
  elif photonAlice[i] == 'V':
    photonArray[i].prepareV(p)
  elif photonAlice[i] == 'D':
    photonArray[i].prepareD(p)
  else:
    photonArray[i].prepareA(p)
print(photonArray)    


[<__main__.Photon object at 0x7f81a6c25e10>, <__main__.Photon object at 0x7f81a6c25c50>, <__main__.Photon object at 0x7f81a6c25290>, <__main__.Photon object at 0x7f81a6c25c90>, <__main__.Photon object at 0x7f81a6c25490>, <__main__.Photon object at 0x7f81a6c25a50>, <__main__.Photon object at 0x7f81a6c25390>, <__main__.Photon object at 0x7f81a6c25610>, <__main__.Photon object at 0x7f81a6c25910>, <__main__.Photon object at 0x7f81a6c25110>, <__main__.Photon object at 0x7f81a6c25710>, <__main__.Photon object at 0x7f81a6c258d0>, <__main__.Photon object at 0x7f81a6c25890>, <__main__.Photon object at 0x7f81a6c254d0>, <__main__.Photon object at 0x7f81a6c25750>, <__main__.Photon object at 0x7f81a6c25850>, <__main__.Photon object at 0x7f81a6c25950>, <__main__.Photon object at 0x7f81a6c25450>, <__main__.Photon object at 0x7f81a6c25150>, <__main__.Photon object at 0x7f81a6c25090>, <__main__.Photon object at 0x7f81a6c25050>, <__main__.Photon object at 0x7f81a6c25a90>, <__main__.Photon object at 0x7f

In [ ]:
# Eve   --------------------------------------------

# You should implement this section after completing Alice and Bob.
# Eve is allowed to do whatever she wants to the photonAlice array.
# She cannot, however, have knowledge of Alice's or Bob's choice of bases, 
# nor Bob's measurement outcomes, until they are publicly announced.

In [ ]:
# Eve selects a subsample of photons from Alice to measure.
# interceptIndex should be a string of n characters.
# Use the convention '0'=ignored, '1'=intercepted
interceptIndex = ""
# TODO: Put your code here.
interceptIndex = ""
maxIntercept = 14
intercepted = 0
# TODO: Put your code here.
for i in range(n):
  if intercepted < maxIntercept:
    interceptIndex += str(random.randint(0,1))
    if interceptIndex[i] == "1":
      intercepted += 1
  else:
    interceptIndex += "0"

print(interceptIndex)

In [ ]:
# Eve chooses a basis to measure each intercepted photon.
# basisEve should be a string of n characters.
# Use the convention '+'=H/V, 'x'=D/A, ' '=not measured
basisEve = ""
# TODO: Put your code here.
for i in range(n):
  if interceptIndex[i] == "1":
    if random.randint(0,1) == 0:
      basisEve += '+'
    else:
      basisEve += 'x'
  else:
    basisEve += ' '

print("basisEve    = " + basisEve)

In [ ]:
# Eve performs a measurement on each photon.
# outcomeEve should be a string of n characters.
# Use the convention 'H','V','D','A', ' '=not measured
outcomeEve = ""
# TODO: Put your code here.
for i in range(n):
  if basisEve[i] == '+':
    outcomeEve += photonArray[i].measureHV(d)
  elif basisEve[i] == 'x':
    outcomeEve += photonArray[i].measureDA(d)
  else:
    outcomeEve += ' '

outcomeEve = outcomeEve.replace("N"," ")
outcomeEve = outcomeEve.replace("M"," ")  


print("outcomeEve  = " + outcomeEve) 

In [ ]:
# Eve resends photons to Bob.
# Be sure to handle the cases in which Eve gets an invalid measurement.
# TODO: Put your code here.
fauxPhotons = [Photon() for i in range(n)]

for i in range(n):
  if outcomeEve[i] == 'H':
      fauxPhotons[i].prepareH(a)
  elif outcomeEve[i] == 'V':
      fauxPhotons[i].prepareV(a)
  elif outcomeEve[i] == 'D':
      fauxPhotons[i].prepareD(a)
  elif outcomeEve[i] == 'A':
      fauxPhotons[i].prepareA(a)
  else:
    fauxPhotons[i] = photonArray[i]

photonArray = fauxPhotons

In [ ]:

# Change M and N to H,V,D,A a basis reading by manipulating amplitude (Attenuate)


In [ ]:
# Bob   --------------------------------------------

# Bob chooses a basis to measure each photon.
# This should be a string of '+'s and 'x's with '+'=H/V, 'x'=D/A.
basisBob = ""
# TODO: Put your code here.
for i in range(n):
  if random.randint(0,1) == 0:
    basisBob += '+'
  else:
    basisBob += 'x'
print("basisBob    = " + basisBob)
# TODO: Put your code here.

basisBob    = ++xxx++x++xx++xx+++++x+x++xx+x+++++++++x+++xx++x++x+++x+++++x++++x++x+x++xxx+xx++x++x+++xx++xxx+x+x+


In [ ]:
# Bob performs a measurement on each photon.
# Use the methods of the Photon class to measure each photon.
# outcomeBob should be a string of n characters.
# Use the convention 'H','V','D','A', ' '=not measured
outcomeBoblist = []
outcomeBob = ""
# TODO: Put your code here.
for i in range(n):
  if basisBob[i] == '+':
    outcomeBoblist.append(photonArray[i].measureHV(d))
  else:
    outcomeBoblist.append(photonArray[i].measureDA(d))

for i in range(n):
  if outcomeBoblist[i] == "N" or outcomeBoblist[i] == "M":
    outcomeBoblist[i] = " "
  outcomeBob += outcomeBoblist[i]


print("outcomeBob  = " + outcomeBob) 

outcomeBob  =    D    VVDDV   HVH   HAHV  HDV  V    HA  VD V  V D   A V  V       V VAV  A     HA V HHV  HHAA HA  V


In [ ]:
# Bob infers the raw key.
keyBob = ""
# TODO: Put your code here.
for i in range(n):
  if basisBob[i] == '+' and outcomeBob[i] == 'H':
    keyBob += '0'
  elif basisBob[i] == '+' and outcomeBob[i] == 'V':
    keyBob += '1'
  elif basisBob[i] == 'x' and outcomeBob[i] == 'D':
    keyBob += '0'
  elif basisBob[i] == 'x' and outcomeBob[i] == 'A':
    keyBob += '1'
  else:
    keyBob += "-"
  
print("keyBob      = " + keyBob)
# Only about half of Bob's raw key with match Alice's raw key.

keyBob      = ---0----11001---010---0101--001--1----01--10-1--1-0---1-1--1-------1-111--1-----01-1-001--0011-01--1


In [ ]:
# -----------------------------------------------------------
# Alice and Bob now publicly announce which bases they chose.
# Bob also announces which of his measurements were invalid.
# -----------------------------------------------------------

In [ ]:
# Alice & Bob ----------------------------------------------------------

# Alice and Bob extract their sifted keys.
# siftedAlice and siftedBob should be strings of length n.
# Use the convention '0', '1', ' '=removed
siftedAlice = ""
siftedBob   = ""
# TODO: Put your code here.

for i in range(n):
  if basisBob[i] == basisAlice[i] and keyBob[i] != "-":
    siftedAlice += keyAlice[i]
    siftedBob += keyBob[i]
  else:
    siftedAlice += ' '
    siftedBob += ' '
  
print("Sifted Alice: " + siftedAlice)
print("Sifted Bob  : " + siftedBob)

Sifted Alice:         11 0     10   010   1             1     1 0   1 1                            101     1 01   
Sifted Bob  :         11 0     10   010   0             1     1 0   1 1                            001     1 01   


In [ ]:
# Alice and Bob use a portion of their sifted keys to estimate the quantum bit error rate (QBER).
# sampleIndex should be a string of n characters.
# Use the convention '0'=ignored, '1'=sampled
# The QBER is the fraction of mismatches within the sampled portion.
# For large samples, it should be close to the actual QBER,
# which Alice and Bob, of course, do not know.
sampleIndex = ""
sampledBobQBER = 0
sampledTotal = 0
sampleMax = 20
for i in range(n):
  if siftedAlice[i] != ' ' and sampledTotal < sampleMax:
    sampleIndex += str(random.randint(0,1))
    if sampleIndex[i] == "1":
      sampledTotal += 1
  else:
    sampleIndex += "0"
  
for i in range(n):
  if sampleIndex[i] == "1":
    if siftedBob[i] == siftedAlice[i]:
      sampledBobQBER += 1
    

sampledBobQBER = 1 - sampledBobQBER/sampledTotal
print (sampledBobQBER)
print (str(len(sampleIndex)))
print (sampledTotal)



0.2857142857142857
100
7


In [ ]:
# Alice and Bob remove the portion of their sifted keys that was sampled.
# Since a portion of the sifted key was publicly revealed, it cannot be used.
# secureAlice and secureBob should be strings of length n.
# Use the convention '0', '1', ' '=removed
secureAlice = ""
secureBob = ""

# TODO: Put your code here.
for i in range(n):
  if sampleIndex[i] == "0":
    secureAlice += siftedAlice[i]
    secureBob += siftedBob[i]
  else:
    secureAlice += " "
    secureBob += " "

print("secureAlice = " + secureAlice)
print("secureBob =   " + secureBob)

secureAlice =         1  0     10   0 0                 1     1 0   1 1                              1       0    
secureBob =           1  0     10   0 0                 1     1 0   1 1                              1       0    


In [ ]:
# Alice and Bob make a hard determination whether the channel is secure.
# If it looks like there's something fishy, better hit the kill switch!
channelSecure = True # default value, to be changed to False if Eve suspected
# TODO: Put your code here.
if sampledBobQBER > 0.05:
  channelSecure = False

In [ ]:
# Eve ------------------------------------------------------------------

In [ ]:
# Eve infers the raw key.
# keyEve should be a string of n characters.
# Use the convention '0', '1', '-'=invalid measurement, ' '=not measured
keyEve = ""
# TODO: Put your code here.
for i in range(n):
  if basisEve[i] == '+' and outcomeEve[i] == 'H':
    keyEve += '0'
  elif basisEve[i] == '+' and outcomeEve[i] == 'V':
    keyEve += '1'
  elif basisBob[i] == 'x' and outcomeEve[i] == 'D':
    keyEve += '0'
  elif basisEve[i] == 'x' and outcomeEve[i] == 'A':
    keyEve += '1'
  elif basisEve[i] == " ":
    keyEve += " "
  else:
    keyEve += "-"
  
print("keyEve= " + keyEve)

In [ ]:
# Eve extracts her sifted key.
# Knowing what Alice and Bob have publically revealed, Eve
# now selects which portion of her sifted key to keep.
# stolenEve should be strings of length n.
# Use the '0', '1', ' '=removed
stolenEve = ""
# TODO: Put your code here.
for i in range(n):
  if basisEve[i] == basisAlice[i]:
    if keyEve[i] != "-":
      stolenEve += keyEve[i]
    else:
      stolenEve += " "
  else:
    stolenEve += " "
  
print("stolenEve= " + stolenEve)

In [ ]:
# ANALYSIS -------------------------------------------------------------

# Below is a standard set of metrics to evaluate each protocol.
# You need not change any of what follows.

# Compare Alice and Bob's sifted keys.
numMatchBob = 0
actualBobQBER = 0
secureKeyRateBob = 0
secureKeyLengthBob = 0
for i in range(len(secureAlice)):
    if secureAlice[i] != ' ':
       secureKeyLengthBob += 1
       if secureAlice[i] == secureBob[i]:
           numMatchBob += 1

# Compute the actual quantum bit error rate for Bob.
if secureKeyLengthBob > 0:
    actualBobQBER = 1 - numMatchBob / secureKeyLengthBob
else:
    actualBobQBER = float('nan')
# Compute the secure key rate, assuming each trial takes 1 microsecond.
secureKeyRateBob = (1-actualBobQBER) * secureKeyLengthBob / n * 1e6;

# Compare Alice and Eve's sifted keys.
numMatchEve = 0
actualEveQBER = 0
stolenKeyRateEve = 0
stolenKeyLengthEve = 0
for i in range(len(stolenEve)):
    if stolenEve[i] != ' ':
       stolenKeyLengthEve += 1
       if secureAlice[i] == stolenEve[i]:
           numMatchEve += 1
# Compute the actual quantum bit error rate for Eve.
if stolenKeyLengthEve > 0:
    actualEveQBER = 1 - numMatchEve / stolenKeyLengthEve
else:
    actualEveQBER = float('nan')
# Compute the stolen key rate, assuming each trial takes 1 microsecond.
stolenKeyRateEve = (1-actualEveQBER) * stolenKeyLengthEve / n * 1e6;


# DISPLAY RESULTS ------------------------------------------------------

print("")
print("basisAlice  = " + basisAlice)
print("basisBob    = " + basisBob)
print("basisEve    = " + basisEve)
print("")
print("keyAlice    = " + keyAlice)
print("keyBob      = " + keyBob)
print("keyEve      = " + keyEve)
print("")
print("siftedAlice = " + siftedAlice)
print("siftedBob   = " + siftedBob)
print("")
print("secureAlice = " + secureAlice)
print("secureBob   = " + secureBob)
print("stolenEve   = " + stolenEve)
print("")
if not channelSecure:
    secureKeyRateBob = 0;
    stolenKeyRateEve = 0;
    print("*********************************************")
    print("* ALERT! The quantum channel is not secure. *")
    print("*********************************************")
    print("")
print("sampledBobQBER = " + str(sampledBobQBER))
print("actualBobQBER  = " + str(actualBobQBER))
print("actualEveQBER  = " + str(actualEveQBER))
print("")
print("secureKeyRateBob = " + str(secureKeyRateBob/1000) + " kbps")
print("stolenKeyRateEve = " + str(stolenKeyRateEve/1000) + " kbps")

# Your goal is to maximize secureKeyRateBob and minimize stolenKeyRateEve.

NameError: ignored